<a href="https://colab.research.google.com/github/abxda/INFOTEC-Aprendizaje-No-Supervisado/blob/main/Gu%C3%ADa_de_Apoyo_Act_01_K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Guía de Apoyo: K-Means para cuantización de colores (Datos Sintéticos)

En esta guía veremos las herramientas de programación necesarias para agrupar datos similares. Imaginen que tenemos una "imagen falsa" generada aleatoriamente. Nuestro objetivo es reducir la variedad de datos a solo unos pocos grupos representativos (clusters).

#### 1. Generación de datos y manejo de dimensiones (Reshape)

Las imágenes en Python se representan como matrices de 3 dimensiones: `(Alto, Ancho, Canales)`. Sin embargo, `KMeans` de scikit-learn necesita una tabla de 2 dimensiones: `(Número de muestras, Características)`.

**Ejemplo:**
Tenemos una imagen pequeña de 100x100 pixeles.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Crear "imagen" sintética (ruido aleatorio)
# Dimensiones: 100 de alto, 100 de ancho, 3 canales (RGB)
alto, ancho = 100, 100
imagen_sintetica = np.random.randint(0, 255, (alto, ancho, 3))

print(f"Dimensiones originales de la imagen: {imagen_sintetica.shape}")
# Salida esperada: (100, 100, 3)

# 2. Aplanar la imagen (Reshape)
# Para K-means, necesitamos una lista larga de píxeles, no una cuadrícula.
# Convertimos (100, 100, 3) -> (10000, 3)
# -1 indica a numpy que calcule esa dimensión automáticamente
pixeles = imagen_sintetica.reshape(-1, 3)

print(f"Dimensiones para K-Means: {pixeles.shape}")
# Salida esperada: (10000, 3)

#### 2. Aplicando K-Means

Una vez que tenemos los datos en el formato `(N_muestras, 3)`, podemos pedirle al algoritmo que encuentre los grupos.

In [ ]:
from sklearn.cluster import KMeans

# Definimos cuántos colores finales queremos (k)
k = 5

# Configuramos el modelo
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42, init='k-means++')

# Entrenamos el modelo con nuestros pixeles aplanados
kmeans.fit(pixeles)

# OBTENCIÓN DE DATOS CLAVE:

# 1. Los "Centroides": Son los colores promedio encontrados
centroides = kmeans.cluster_centers_
print(f"\nCentroides (Colores principales):\n {centroides}")

# 2. Las "Etiquetas" (Labels): Dice a qué grupo pertenece cada pixel original
etiquetas = kmeans.labels_
print(f"\nEtiquetas (primeros 10): {etiquetas[:10]}")
print(f"Cantidad total de etiquetas: {etiquetas.shape}")

#### 3. Reconstrucción de la imagen (El paso mágico)

Ahora que sabemos qué etiqueta tiene cada pixel (0, 1, 2, etc.) y qué color real representa esa etiqueta (los centroides), debemos reemplazar los datos originales por los colores de los centroides.

**Truco de NumPy:** Podemos usar las etiquetas como índices para seleccionar los colores de la matriz de centroides.

In [ ]:
# Asignamos a cada pixel el color de su centroide correspondiente
# Esto reemplaza el valor original por el valor del centroide
pixeles_comprimidos = centroides[etiquetas]

# IMPORTANTE: K-means devuelve floats, pero las imágenes suelen ser enteros (uint8)
pixeles_comprimidos = pixeles_comprimidos.astype("uint8")

print(f"Forma de datos comprimidos: {pixeles_comprimidos.shape}")

# Devolvemos la forma original de imagen (Alto, Ancho, 3) para poder visualizarla
imagen_final = pixeles_comprimidos.reshape(alto, ancho, 3)
print(f"Forma de imagen final reconstruida: {imagen_final.shape}")

#### 4. Visualización

Comparación entre los datos aleatorios originales y los datos agrupados por K-Means.


In [ ]:
# Visualizar
plt.figure(figsize=(10, 5))

# Imagen original (ruido puro)
plt.subplot(1, 2, 1)
plt.title("Datos Originales (Aleatorios)")
plt.imshow(imagen_sintetica)
plt.axis('off')

# Imagen cuantizada (reducida a k colores)
plt.subplot(1, 2, 2)
plt.title(f"Datos Agrupados (K={k})")
plt.imshow(imagen_final)
plt.axis('off')

plt.show()

In [ ]:
from PIL import Image
import io

def obtener_peso_en_bytes(arr_imagen):
    arr_imagen = arr_imagen.astype("uint8")

    img = Image.fromarray(arr_imagen)
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    return buffer.tell()

# Calculamos los pesos
peso_original = obtener_peso_en_bytes(imagen_sintetica)
peso_kmedias = obtener_peso_en_bytes(imagen_final)

print(f"--- Resultados de Compresión (Formato PNG) ---")
print(f"Peso Imagen Original (Ruido): {peso_original / 1024:.2f} KB")
print(f"Peso Imagen K-Means (k={k}):   {peso_kmedias / 1024:.2f} KB")

ahorro = 100 * (1 - (peso_kmedias / peso_original))
print(f"¡Reducción de tamaño del {ahorro:.1f}%!")

### ¿Por qué se redujo el tamaño del archivo?

La reducción de peso se debe al aumento de la **redundancia**.

1. **Menos Variedad:** La imagen original (ruido aleatorio) tenía miles de colores distintos, donde cada pixel era diferente a su vecino. Esto es "alta entropía" o mucha información única.
2. **Agrupamiento (K-Means):** Al aplicar K-Means, forzamos a toda la imagen a usar solo ** colores** (ej. 5 o 8).
3. **Compresión Inteligente (PNG):** Los formatos de imagen como PNG funcionan detectando patrones.
* En la imagen original, no hay patrones; el archivo debe guardar el color de cada pixel individualmente.
* En la imagen procesada, muchos pixeles vecinos ahora comparten el *exacto mismo color*. El formato PNG puede decir: *"Los siguientes 50 pixeles son del color A"* en lugar de escribir 50 veces el código del color.



**En resumen:** K-Means simplificó la información visual, creando patrones repetitivos que la computadora puede guardar de forma mucho más eficiente.